In [ ]:
HTML(read(open("style.css"), String))

# Quiescence Search

Quiescence Search is a search algorithm designed to avoid the horizon effect in the game of chess. It is called when the search function (e.g. Minimax, alpha-beta-pruning,...) have reached their maximum depth. Quiescence Search will then further search through the game tree until it hits a quiet position. 

A position is quiet if: $no\ captures (en\ passant\ included) \vee no\ checks \vee no\ promotions$

Additionally a position is tactical if it is not quiet. Accordingly a tactical move is a move where a piece is captured, a check is given or a pawn is promoted.

Then and only then it will use the heuristic function to estimate the position. It is important to not stop searching when hitting the end of the depth as this last move can conclude into very good moves for the opposing side. As an example the last move searched by the algorithm is a queen capturing a pawn. The evaluation function therefore will return a score of +100 centipawns as it just took this pawn. Notice that the search algorithm will not see that this pawn was protected resulting in the queen being captured in the move after. This issue is called the **horizon effect**.

Quiescence Search will therefore keep calculate all tactical moves and stop when it hits a quiet position. As only the tactical moves will be evaluated, this evaluation will not take as much performance as a regular search. Additionally Pruning will result in about 50-90% of paths pruned. (https://www.chessprogramming.org/Quiescence_Search)

There are many different approaches on implementing Quiescence Search. The definition of a quiet position also varies. Therefore we define a quiet position as follows:

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using NBInclude
using BenchmarkTools

[Chess Programming Wiki "Quiescence Search"](https://www.chessprogramming.org/Quiescence_Search)

In [ ]:
@nbinclude("AdvancedBoard.ipynb")

In [ ]:
@nbinclude("Memoization.ipynb")

## Auxiliary functions

### Function: hasCaptureMoves
This function takes in a chess game state and returns a boolean indicating whether there are any capture moves available for the opponent.

Arguments:
1. `State::Board` A chess board in the current state.

Returns whether there are any capture moves available for the opponent.

In [ ]:
function hasCaptureMoves(State::Board)
    allPiecesSq = pieces(State, coloropp(sidetomove(State)))
    return any(isattacked(State, square, coloropp(sidetomove(State))) for square in allPiecesSq)
end

### Function: isTacticalMove
The function `isTacticalMove` checks whether the move done on a state is a tactical move. 

A move is a tactical move if is one of the following is true:
1. The move is capturing a piece
1. The move is promoting a pawn
1. The move is an en passant capture
1. The move checks the opposing king

Arguments:

1. `State::Board` The current board
1. `move::Move`   The move on the `State` that should be checked for tactical moves

Returns:

1. `true` if the move is a tactical move, otherwise false

In [ ]:
function isTacticalMove(State::Board, move::Move)
    if pieceon(State, to(move)) != EMPTY || ispromotion(move) || epsquare(State) == to(move)
        return true
    end
    undoinfo = domove!(State, move)
    isNextMoveCheck = ischeck(State)
    undomove!(State, undoinfo)
    return isNextMoveCheck
end

## Quiescence Min and Max

Due to performance issues we had to simplify the quiescence search. The amount of capturing moves during the middlegame of chess is very high resulting in a large game tree when using the quiescence search. During developement we reached positions about 20 plys deeper after starting the quiescence search. This lead to long calculation times. The simplified version of quiescence search is limited to a maximum of 3. This means that when the search-algorithm has reached its maximum depth the quiescence search will do an additional 3 steps if necessary. After that it will return the static evaluation of the board for this position. 

The function `quiescenceMax` uses the quiescence algorithm to determine the best score (meaning maximizing the score) for positions where white moves. It only stops searching if the position is quiet and only searches through positions which have tactical moves.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `hash::UInt64` The hash of the current board
1. `depth::Int64` The depth of the search (initially always set to 0 and decremented by each step)
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceMax(aBoard::AdvBoard, depth::Int64, alpha::Int64, beta::Int64, )::Int64
    value::Int64 = aBoard.score
    depth <= -4 && return aBoard.score
    value >= beta && return beta
    alpha = max(alpha, value)

    for move in moves(aBoard.state)
        isTacticalMove(aBoard.state, move) || continue
        undoinfo::Tuple{Int64, UInt64, UndoInfo} = domoveAdv!(aBoard, move)
        value = quiesceMin(aBoard, depth-1, alpha, beta)
        undomoveAdv!(aBoard, undoinfo)
        value >= beta && return beta
        alpha = max(alpha, value) 
    end
    return alpha
end

The function `quiescenceMin` uses the quiescence algorithm to determine the best score (meaning minimizing the score) for positions where black moves. It only stops searching if the position is quiet and only searches through positions which have tactical moves.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `hash::UInt64` The hash of the current board
1. `depth::Int64` The depth of the search (initially always set to 0 and decremented by each step)
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceMin(aBoard::AdvBoard, depth::Int64, alpha::Int64, beta::Int64)::Int64
    value = aBoard.score
    depth <= -4 && return aBoard.score
    value <= alpha && return alpha
    beta = min(value, beta)

    for move in moves(aBoard.state)
        isTacticalMove(aBoard.state, move) || continue
        undoinfo::Tuple{Int64, UInt64, UndoInfo} = domoveAdv!(aBoard, move)
        value = quiesceMax(aBoard, depth-1, alpha, beta)
        undomoveAdv!(aBoard, undoinfo)
        value <= alpha && return alpha
        beta = min(value, beta)
    end
    return beta
end

## Quiescence Search with see() function

The function `quiesceSee` uses the `see()` function from `Chess.jl` to determine an estimate whether any pieces may be captured in the near future. The see function sums captured pieces values in piece points. For example the see function returns that 5 points of material was lost when a rook is captured. Therefore this value is multiplied by 100 as this roughly delivers the centipawn value.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceSee(aBoard::AdvBoard, alpha::Int64, beta::Int64)
    if sidetomove(aBoard.state) == WHITE
        bestEstimate::Int64 = -100000
        for move in moves(aBoard.state)
            bestEstimate = max(bestEstimate, see(aBoard.state, move))
        end
    else 
        bestEstimate = 100000
        for move in moves(aBoard.state)
            bestEstimate = min(bestEstimate, see(aBoard.state, move))
        end
    end

    return aBoard.score + bestEstimate * 100
end